In [21]:
import config
from bs4 import BeautifulSoup
import requests
import pandas as pd
import spotipy as sp
import pprint
from spotipy.oauth2 import SpotifyClientCredentials
import json
from IPython.display import display, FileLink
import numpy as np
import pickle
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.metrics import silhouette_score
import random
from IPython.display import IFrame

In [22]:
with open("kmeans_14.pickle", "rb") as f:
    scaler = pickle.load(f)

def load(filename = "kmeans_14.pickle"): 
    try: 
        with open(filename, "rb") as f: 
            return pickle.load(f) 
        
    except FileNotFoundError: 
        print("File not found!") 

In [23]:
#Initialize SpotiPy with user credentials
sp = sp.Spotify(auth_manager=SpotifyClientCredentials(client_id= config.client_id,
                                                           client_secret= config.client_secret))

In [24]:
pottify_df_clustered = pd.read_csv('pottify_df_clustered.csv')
pottify_df_clustered.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1193 entries, 0 to 1192
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   danceability      1193 non-null   float64
 1   energy            1193 non-null   float64
 2   key               1193 non-null   float64
 3   loudness          1193 non-null   float64
 4   mode              1193 non-null   float64
 5   speechiness       1193 non-null   float64
 6   acousticness      1193 non-null   float64
 7   instrumentalness  1193 non-null   float64
 8   liveness          1193 non-null   float64
 9   valence           1193 non-null   float64
 10  tempo             1193 non-null   float64
 11  cluster           1193 non-null   float64
 12  id                1193 non-null   object 
dtypes: float64(12), object(1)
memory usage: 121.3+ KB


In [31]:
def recommended_songs (user_input):

    user_input = input("Enter a song title: ").lower()
    
    #get song ID based on user input
    api_request = sp.search(q = user_input, limit=1)
    song_id = str(api_request["tracks"]["items"][0]["id"])

    #get features of song
    song_id_af = sp.audio_features(song_id)
    
    # make dataframe of AF
    user_input_df = pd.DataFrame(song_id_af)
    user_input_df.drop(columns = ['type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms', 'time_signature'], inplace = True)

    #scale the user's dataframe
    user_input_df_scaled = scaler.transform(user_input_df)

    #predict the cluster based on user df
    user_song_cluster_pred = scaler.predict(user_input_df)
    
    #recieve cluster
    user_song_cluster = user_song_cluster_pred[0]
    
    #create a new dataframe based on predicted cluster
    user_song_cluster_df = pottify_df_clustered[pottify_df_clustered['cluster'] == user_song_cluster]
    user_song_cluster_df = user_song_cluster_df.reset_index(drop=True)     
    
    #apply model to dataframe
    random_songs_indexed = random.randint(0, len(user_song_cluster_df) - 1)  
    random_songs_id = user_song_cluster_df['id'].iloc[random_songs_indexed]

    print("Here is a recommendation based on the song you entered:")
    return IFrame(src="https://open.spotify.com/embed/track/" + random_songs_id,
              width="320",
              height="80",
              frameborder="0",
              allowtransparency="true",
              allow="encrypted-media")

In [34]:
# Call the function with the sample input
recommendation_iframe = recommended_songs(user_input)

# Display the recommendation iframe
recommendation_iframe


Enter a song title: sweet caroline
Here is a recommendation based on the song you entered:
